### 설치

In [ ]:
# konlpy 설치
!pip install konlpy

In [ ]:
# 라이브러리

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
okt = Okt()
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re


from tqdm.notebook import tqdm

import pandas as pd
import requests
from bs4 import BeautifulSoup

### 데이터 로드 및 전처리(전처리 코드 코랩과 동일)

In [ ]:
data = pd.read_excel('/content/크롤링완료.xlsx')
data

,위원회,사건번호,판정일,결과,내용
0,11,1999부해21,9199-05-31,NaN,NaN
1,10,2021부해337,2022-04-13,전부인정,"원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점, 이 사건..."
2,12,2022부해26,2022-04-06,전부인정,가. 징계처분 1의 정당성 근로자가 2차례(2021. 5. 24. 및 2021....
3,1,2022부해272,2022-04-01,전부인정,가. 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...
4,1,2022부해266,2022-03-31,각하,① 근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...
...,...,...,...,...,...
55705,2,2019부해522,--,각하,NaN
55706,0,2019부해1411,--,초심취소,NaN
55707,1,2019부해2972,--,기각,NaN
55708,8,2019부해623,--,기각,NaN


In [ ]:
new_data = data.dropna(inplace=False)
data['위원회'].count(), new_data['위원회'].count()

(55710, 43688)

In [ ]:
new_data = new_data[new_data['위원회']>0]
new_data

,위원회,사건번호,판정일,결과,내용
1,10,2021부해337,2022-04-13,전부인정,"원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점, 이 사건..."
2,12,2022부해26,2022-04-06,전부인정,가. 징계처분 1의 정당성 근로자가 2차례(2021. 5. 24. 및 2021....
3,1,2022부해272,2022-04-01,전부인정,가. 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...
4,1,2022부해266,2022-03-31,각하,① 근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...
5,11,2022부해18,2022-03-31,기각,"1. 상시근로자 수가 5명 이상인지 여부 ① 사업주가 서로 다른 개인인 점, ② ..."
...,...,...,...,...,...
53698,3,2014부노17,--,기각,가. 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...
54445,2,2001부해192,--,각하,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...
54447,9,2001부해53,--,기각,"강박된 상태에서 사직서를 제출하지 아니 하였고, 사직서가 제출되면 수리될 수도 있음..."
54482,2,2001부해120,--,전부인정,[피신청인 성명]전 정 도[피신청인 사업체명]성진지오텍(주)[신청요지]1. 본건 신...


In [ ]:
new_data['내용'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s)for s in new_data['내용']]
new_data.reset_index(inplace=True)
new_data

,index,위원회,사건번호,판정일,결과,내용
0,1,10,2021부해337,2022-04-13,전부인정,원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점 이 사건...
1,2,12,2022부해26,2022-04-06,전부인정,가 징계처분 1의 정당성 근로자가 2차례 2021 5 24 및 2021 ...
2,3,1,2022부해272,2022-04-01,전부인정,가 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...
3,4,1,2022부해266,2022-03-31,각하,근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...
4,5,11,2022부해18,2022-03-31,기각,1 상시근로자 수가 5명 이상인지 여부 사업주가 서로 다른 개인인 점 ...
...,...,...,...,...,...,...
31288,53698,3,2014부노17,--,기각,가 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...
31289,54445,2,2001부해192,--,각하,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...
31290,54447,9,2001부해53,--,기각,강박된 상태에서 사직서를 제출하지 아니 하였고 사직서가 제출되면 수리될 수도 있음...
31291,54482,2,2001부해120,--,전부인정,피신청인 성명 전 정 도 피신청인 사업체명 성진지오텍 주 신청요지 1 본건 신...


In [ ]:
new_data.loc[new_data['결과'].str.contains('기각'), '라벨링'] = 0
new_data.loc[new_data['결과'].str.contains('전부인정'), '라벨링'] = 1
new_data.loc[new_data['결과'].str.contains('각하'), '라벨링'] = 2

new_data.dropna(inplace=True)

In [ ]:
new_data['라벨링'] = new_data['라벨링'].astype(int)

In [ ]:
new_data.reset_index(drop=True,inplace=True)
new_data

,index,위원회,사건번호,판정일,결과,내용,라벨링
0,1,10,2021부해337,2022-04-13,전부인정,원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점 이 사건...,1
1,2,12,2022부해26,2022-04-06,전부인정,가 징계처분 1의 정당성 근로자가 2차례 2021 5 24 및 2021 ...,1
2,3,1,2022부해272,2022-04-01,전부인정,가 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...,1
3,4,1,2022부해266,2022-03-31,각하,근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...,2
4,5,11,2022부해18,2022-03-31,기각,1 상시근로자 수가 5명 이상인지 여부 사업주가 서로 다른 개인인 점 ...,0
...,...,...,...,...,...,...,...
30145,53698,3,2014부노17,--,기각,가 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...,0
30146,54445,2,2001부해192,--,각하,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...,2
30147,54447,9,2001부해53,--,기각,강박된 상태에서 사직서를 제출하지 아니 하였고 사직서가 제출되면 수리될 수도 있음...,0
30148,54482,2,2001부해120,--,전부인정,피신청인 성명 전 정 도 피신청인 사업체명 성진지오텍 주 신청요지 1 본건 신...,1


In [ ]:
fin_data = {
    '내용':[],
    '라벨':[]
}

In [ ]:
for i in tqdm(range(len(new_data))) :
  if len(list(new_data['내용'].values)[i]) >= 50 :
    fin_data['내용'].append(new_data['내용'][i])
    fin_data['라벨'].append(new_data['라벨링'][i])

fin_data = pd.DataFrame(fin_data)
fin_data

  0%|          | 0/30150 [00:00<?, ?it/s]

,내용,라벨
0,원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점 이 사건...,1
1,가 징계처분 1의 정당성 근로자가 2차례 2021 5 24 및 2021 ...,1
2,가 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...,1
3,근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...,2
4,1 상시근로자 수가 5명 이상인지 여부 사업주가 서로 다른 개인인 점 ...,0
...,...,...
29580,본 건 신청에 대하여 우리위원회는 당사자들의 주장 심문내용 관련자료 ...,0
29581,가 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...,0
29582,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...,2
29583,강박된 상태에서 사직서를 제출하지 아니 하였고 사직서가 제출되면 수리될 수도 있음...,0


### 토큰화+벡터화

In [ ]:
def tokenizer(raw, pos=["Noun", "Verb","Adjective" ] , stopword = []):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=2,
    sublinear_tf = True
)

In [ ]:
x = vectorize.fit_transform(fin_data['내용'])

In [ ]:
y = fin_data['라벨']

#교차검증 (case 10)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
def make_df(x):
  result = pd.DataFrame(x,index = ['case1', 'case2', 'case3', 'case4', 'case5', 'case6', 'case7', 'case8', 'case9', 'case10'])
  mean = pd.DataFrame(result.mean()).T
  return result.append(mean)

####간단한 요약

In [ ]:
clf = LogisticRegression()
model_rf = RandomForestClassifier()
svc = LinearSVC()
NB_clf = MultinomialNB()

score1 = cross_val_score(clf, x, y, cv=10 )
score2 = cross_val_score(model_rf, x, y, cv=10 )
score3 = cross_val_score(svc, x, y, cv=10 )
score4 = cross_val_score(NB_clf, x, y, cv=10 )

print('로지스틱 회귀 교차검증 점수 : {:.2f}'.format(score1.mean()))
print('랜덤포레스트 교차검증 점수 : {:.2f}'.format(score2.mean()))
print('SVC 교차검증 점수 : {:.2f}'.format(score3.mean()))
print('나이브 베이즈 교차검증 점수 : {:.2f}'.format(score4.mean()))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


로지스틱 회귀 교차검증 점수 : 0.85
랜덤포레스트 교차검증 점수 : 0.83
SVC 교차검증 점수 : 0.84
나이브 베이즈 교차검증 점수 : 0.75


####로지스틱 회귀 모델

In [ ]:
#로지스틱 회귀 모델 교차검증
from sklearn.model_selection import cross_validate
clf = LogisticRegression()
res = cross_validate(clf, x, y, return_train_score=True, cv=10)

make_df(res)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


,fit_time,score_time,test_score,train_score
case1,8.265670,0.003206,0.824265,0.899459
case2,8.163295,0.007939,0.860088,0.898107
case3,8.216238,0.007941,0.867185,0.895966
case4,8.036845,0.003418,0.856370,0.898558
case5,8.167770,0.003331,0.860764,0.898220
case6,8.667962,0.003390,0.864097,0.898637
case7,8.140724,0.008577,0.866126,0.897510
case8,7.886212,0.003548,0.850913,0.899463
case9,8.588722,0.006678,0.824544,0.901378
case10,7.602469,0.008904,0.807978,0.902317


#### 랜덤 포레스트 모델

In [ ]:
# 랜덤 포레스트 교차검증 
model_rf = RandomForestClassifier()
re = cross_validate(model_rf, x, y, return_train_score=True, cv=10)
make_df(re)

,fit_time,score_time,test_score,train_score
case1,51.422554,0.177904,0.820547,0.999587
case2,50.877678,0.170618,0.858736,0.999662
case3,51.448254,0.175785,0.846570,0.999624
case4,51.704802,0.184961,0.844542,0.999549
case5,51.183230,0.184540,0.835079,0.999624
case6,51.184137,0.182289,0.846180,0.999587
case7,51.274395,0.182423,0.834348,0.999549
case8,51.791834,0.190140,0.821839,0.999587
case9,51.301553,0.195609,0.822853,0.999549
case10,51.013127,0.206065,0.788709,0.999700


####나이브 베이즈 모델

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
r = cross_validate(NB_clf, x, y, return_train_score=True, cv=10)
make_df(r)


,fit_time,score_time,test_score,train_score
case1,0.032609,0.002064,0.719162,0.791369
case2,0.027074,0.002082,0.748564,0.786975
case3,0.026842,0.002127,0.740453,0.787276
case4,0.027439,0.002274,0.739777,0.788515
case5,0.027390,0.002283,0.748564,0.788966
case6,0.027552,0.002167,0.758959,0.786570
case7,0.028206,0.003641,0.762001,0.787622
case8,0.026415,0.002346,0.766396,0.788711
case9,0.027334,0.002221,0.751183,0.787960
case10,0.027568,0.002291,0.730223,0.792354


####선형 SVC 모델

In [ ]:
# LinearSVC모델 

from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
r = cross_validate(l_svc, x, y, return_train_score=True, cv=10)
make_df(r)


# model = LinearSVC(C=1000)
# model.fit(X_train_tfidf, y_train)
# print(model.score(X_train_tfidf, y_train))
# print(model.score(X_test_tfidf, y_test))
    

,fit_time,score_time,test_score,train_score
case1,1.548965,0.003204,0.821561,0.934350
case2,1.534413,0.002197,0.857722,0.931834
case3,1.474513,0.002054,0.868875,0.930970
case4,1.428682,0.002110,0.851977,0.931871
case5,1.420909,0.001988,0.852991,0.931646
case6,1.458678,0.002037,0.849899,0.931085
case7,1.422014,0.002143,0.852265,0.931836
case8,1.436174,0.002280,0.842461,0.932850
case9,1.432919,0.002428,0.819811,0.934014
case10,1.420361,0.002160,0.798513,0.934916


#### SVC 모델

In [ ]:
#SVC모델 
svc = SVC()
rr = cross_validate(svc, x, y, return_train_score=True, cv=10)
make_df(rr)


,fit_time,score_time,test_score,train_score
case1,815.220599,42.031967,0.841838,0.977128
case2,883.655045,40.462403,0.883745,0.974912
case3,889.292051,41.514455,0.883407,0.974987
case4,875.300840,42.639492,0.874958,0.974912
case5,879.427814,43.155855,0.880027,0.974311
case6,901.742134,42.526902,0.884381,0.974762
case7,882.862612,43.725509,0.873901,0.975063
case8,902.754027,44.233040,0.861393,0.975626
case9,913.316760,42.574889,0.844828,0.976190
case10,883.470532,41.921808,0.816430,0.977091
